In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from IPython.display import clear_output
import time
import catboost
import re
import optuna
import sys
sys.path.append('../..')
import main

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin

In [34]:
import m3

In [2]:
data = pd.read_csv(r"D:\Intellipaat\kegal datasets\steel plates dataset\train.csv")
data

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19214,19214,749,757,143210,143219,17,4,4,2193,122,...,-0.1429,0.0044,0.2901,0,0,0,1,0,0,0
19215,19215,723,735,2488529,2488541,231,17,26,27135,104,...,0.7222,-0.0989,0.5378,0,0,0,0,0,0,1
19216,19216,6,31,1578055,1578129,780,114,98,71112,41,...,0.7719,-0.4283,0.9997,1,0,0,0,0,0,0
19217,19217,9,18,1713172,1713184,126,13,26,14808,88,...,0.9610,-0.1162,0.3509,0,0,0,0,0,0,1


In [3]:
col_name = []
with open(r"D:\Intellipaat\kegal datasets\steel plates dataset\Faults27x7_var") as f:
    for line in f:
        col_name.append(line.strip())

In [4]:
df = data.drop(columns=['id'])

#### for now don't use original data, just use train.csv

In [5]:
# org = pd.read_csv("D:\Intellipaat\kegal datasets\steel plates dataset\Faults.NNA",engine= "python",delimiter= '\s',names= col_name)
# org

In [6]:
# df = pd.concat([data,org],ignore_index= True)
# df

In [7]:
X = df.iloc[:,:-7]

In [8]:
pastry = df['Pastry'].copy()
z_scratch = df['Z_Scratch'].copy()
k_scatch = df['K_Scatch'].copy()
stains = df['Stains'].copy()
dirtiness = df['Dirtiness'].copy()
bumps = df['Bumps'].copy()
other_faults = df['Other_Faults'].copy()

In [9]:
Y = [pastry, z_scratch, k_scatch, stains, dirtiness, bumps, other_faults]
Y_col_name = ['pastry', 'z_scratch', 'k_scatch', 'stains', 'dirtiness', 'bumps', 'other_faults']

In [10]:
y_params_dict = {}
for y_nm in Y_col_name:
    y_params_dict[y_nm] = None

In [11]:
y_scores = {}

In [12]:
# cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# for y,yn in zip(Y,Y_col_name):
    
#     print(f'trial: {yn} \n')
    
#     def objective_lgb(trial):
    
#         cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
#         params = dict(
#             n_estimators = trial.suggest_int('n_estimators',100,500),
#             max_depth = trial.suggest_int('max_depth',2,64),
#             num_leaves = trial.suggest_int('num_leaves',2,128),
#             learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
#             min_child_samples = trial.suggest_int('min_child_samples',2,500),
#             min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
#             subsample = trial.suggest_float('subsample', 0.33,0.85),
#             colsample_bytree = trial.suggest_float('colsample_bylevel',0.33,0.7),
#             reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
#             reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
#         )
        
#         lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary',**params)
        
#         pipe = Pipeline(
#             steps = [
#                 ('scaler', StandardScaler()),
#                 ('model', lgbc)
#             ]
#         )
        
#         score = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
#         return score
    
#     study_lgbc = optuna.create_study(direction='maximize')
#     study_lgbc.optimize(objective_lgb, n_trials=1000, n_jobs=17, show_progress_bar=True)
    
#     best_params = study_lgbc.best_params
#     y_params_dict[yn] = best_params
    
#     best_score = study_lgbc.best_value
#     y_scores[yn] = best_score
    
#     time.sleep(5)
#     clear_output()

    
#     y_scores[yn] = f'score: {best_score}'
    
#     with open('lgbc_results.txt', 'w') as file:
#         for tup in y_scores:
#             file.write(f'{tup[0]}, {tup[1]} \n')

In [13]:
y_scores = {}
cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

for y, yn in zip(Y, Y_col_name):
    
    print(f'trial: {yn} \n')
    
    def objective_lgb(trial):
    
        cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        params = dict(
            n_estimators=trial.suggest_int('n_estimators', 100, 500),
            max_depth=trial.suggest_int('max_depth', 2, 64),
            num_leaves=trial.suggest_int('num_leaves', 2, 128),
            learning_rate=trial.suggest_float('learning_rate', 0.001, 0.3),
            min_child_samples=trial.suggest_int('min_child_samples', 2, 500),
            min_child_weight=trial.suggest_float('min_child_weight', 0.01, 10),
            subsample=trial.suggest_float('subsample', 0.33, 0.85),
            colsample_bytree=trial.suggest_float('colsample_bylevel', 0.33, 0.7),
            reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
            reg_lambda=trial.suggest_float('reg_lambda', 0.001, 0.1)
        )
        
        lgbc = lgb.LGBMClassifier(random_state=0, objective='binary', **params)
        
        pipe = Pipeline(
            steps=[
                ('scaler', StandardScaler()),
                ('model', lgbc)
            ]
        )
        
        score = np.mean(cross_val_score(pipe, X, y, scoring='roc_auc', cv=cvo))
        return score
    
    study_lgbc = optuna.create_study(direction='maximize')
    study_lgbc.optimize(objective_lgb, n_trials=1000, n_jobs=18, show_progress_bar=True)
    
    best_params = study_lgbc.best_params
    y_params_dict[yn] = best_params
    
    best_score = study_lgbc.best_value
    y_scores[yn] = best_score
    
    time.sleep(5)
    clear_output()

    with open('lgbc_results.txt', 'w') as file:
        for key, value in y_scores.items():
            file.write(f'{key}, score: {value}\n')


In [14]:
desc = 'using lgbc with hpt'

In [35]:
m3.saver(y_params_dict,y_scores,desc)

In [21]:
y_params_dict

{'pastry': {'n_estimators': 268,
  'max_depth': 7,
  'num_leaves': 49,
  'learning_rate': 0.028653791537353486,
  'min_child_samples': 220,
  'min_child_weight': 8.161362867207485,
  'subsample': 0.760539624867179,
  'colsample_bylevel': 0.3429426696675202,
  'reg_alpha': 0.04055795119180882,
  'reg_lambda': 0.01665388610630641},
 'z_scratch': {'n_estimators': 265,
  'max_depth': 13,
  'num_leaves': 8,
  'learning_rate': 0.03461234781966777,
  'min_child_samples': 255,
  'min_child_weight': 1.365880337903996,
  'subsample': 0.6908677960779537,
  'colsample_bylevel': 0.5138634419662875,
  'reg_alpha': 0.054810677706088065,
  'reg_lambda': 0.08997659015493092},
 'k_scatch': {'n_estimators': 150,
  'max_depth': 26,
  'num_leaves': 102,
  'learning_rate': 0.0423799474213789,
  'min_child_samples': 378,
  'min_child_weight': 3.8317578127103626,
  'subsample': 0.7185052956099005,
  'colsample_bylevel': 0.3366388679181283,
  'reg_alpha': 0.07599238165092129,
  'reg_lambda': 0.0264157672196922

In [30]:
y_scores

{'pastry': 0.8718659338620542,
 'z_scratch': 0.9614309089178693,
 'k_scatch': 0.9862597326047988,
 'stains': 0.9929682379502802,
 'dirtiness': 0.8962142116237377,
 'bumps': 0.8118511010073408,
 'other_faults': 0.7089332756926832}

In [31]:
desc

'using lgbc with hpt'

In [17]:
best_score

0.7089332756926832

In [18]:
y_scores

{'pastry': 0.8718659338620542,
 'z_scratch': 0.9614309089178693,
 'k_scatch': 0.9862597326047988,
 'stains': 0.9929682379502802,
 'dirtiness': 0.8962142116237377,
 'bumps': 0.8118511010073408,
 'other_faults': 0.7089332756926832}

In [19]:
best_params

{'n_estimators': 349,
 'max_depth': 36,
 'num_leaves': 21,
 'learning_rate': 0.01411406314206037,
 'min_child_samples': 264,
 'min_child_weight': 6.2942666521980675,
 'subsample': 0.6133491463732469,
 'colsample_bylevel': 0.5833018776480303,
 'reg_alpha': 0.05266834939382729,
 'reg_lambda': 0.005823354468374447}

In [20]:
test_org = pd.read_csv(r"D:\Intellipaat\kegal datasets\steel plates dataset\test.csv")
test_org

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,19219,1015,1033,3826564,3826588,659,23,46,62357,67,...,0.0095,0.5652,1.0000,1.0,2.8410,1.1139,1.6628,0.6727,-0.2261,0.9172
1,19220,1257,1271,419960,419973,370,26,28,39293,92,...,0.0047,0.2414,1.0000,1.0,2.5682,0.9031,1.4472,0.9063,-0.1453,0.9104
2,19221,1358,1372,117715,117724,289,36,32,29386,101,...,0.0155,0.6000,0.7500,0.0,2.4609,1.3222,1.3222,-0.5238,-0.0435,0.6514
3,19222,158,168,232415,232440,80,10,11,8586,107,...,0.0037,0.8000,1.0000,1.0,1.9031,0.6990,1.0414,0.1818,-0.0738,0.2051
4,19223,559,592,544375,544389,140,19,15,15524,103,...,0.0158,0.8421,0.5333,0.0,2.1461,1.3222,1.1461,-0.5714,-0.0894,0.4170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12809,32028,1101,1116,447943,447992,313,32,37,21603,79,...,0.0126,0.4063,0.9194,1.0,2.4955,1.2305,1.6335,0.7661,-0.3109,0.8894
12810,32029,1289,1306,3149494,3149542,59,9,18,5249,113,...,0.0052,0.7778,1.0000,1.0,1.7708,0.8451,1.2553,0.7222,-0.0448,0.1954
12811,32030,41,210,1587535,1587191,16584,796,522,1858162,24,...,0.1236,0.2199,0.4097,0.0,4.2525,2.2504,2.2672,-0.0629,-0.0801,1.0000
12812,32031,1329,1340,702237,702267,386,43,34,36875,66,...,0.0095,0.2407,1.0000,1.0,2.5866,1.1139,1.5911,0.8461,-0.2629,0.7844


In [22]:
pastry_params = y_params_dict['pastry']
pastry_params

{'n_estimators': 268,
 'max_depth': 7,
 'num_leaves': 49,
 'learning_rate': 0.028653791537353486,
 'min_child_samples': 220,
 'min_child_weight': 8.161362867207485,
 'subsample': 0.760539624867179,
 'colsample_bylevel': 0.3429426696675202,
 'reg_alpha': 0.04055795119180882,
 'reg_lambda': 0.01665388610630641}

In [38]:
pastry_params = y_params_dict['pastry']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [47]:
z_scratch_params = y_params_dict['z_scratch']

z_scratch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**z_scratch_params))
    ]
)
z_scratch_m.fit(X,z_scratch)
z_scratch_pred = z_scratch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [48]:
k_scatch_params = y_params_dict['k_scatch']

k_scatch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**k_scatch_params))
    ]
)
k_scatch_m.fit(X,k_scatch)
k_scatch_pred = k_scatch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [49]:
stains_params = y_params_dict['stains']

stains_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**stains_params))
    ]
)
stains_m.fit(X,stains)
stains_pred = stains_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [50]:
dirtiness_params = y_params_dict['dirtiness']

dirtiness_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**dirtiness_params))
    ]
)
dirtiness_m.fit(X,dirtiness)
dirtiness_pred = dirtiness_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [51]:
bumps_params = y_params_dict['bumps']

bumps_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**bumps_params))
    ]
)
bumps_m.fit(X,bumps)
bumps_pred = bumps_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [52]:
other_faults_params = y_params_dict['other_faults']

other_faults_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**other_faults_params))
    ]
)
other_faults_m.fit(X,other_faults)
other_faults_pred = other_faults_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [53]:
submission = pd.DataFrame({'id':test_org['id'].copy(), 'Pastry':pastry_pred, 'Z_Scratch':z_scratch_pred, 'K_Scatch':k_scatch_pred,
                    'Stains':stains_pred, 'Dirtiness':dirtiness_pred, 'Bumps':bumps_pred, 'Other_Faults':other_faults_pred})

In [54]:
submission

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.578303,0.001583,0.003104,0.000081,0.013791,0.176969,0.342315
1,19220,0.294900,0.012785,0.003421,0.000113,0.178102,0.147274,0.341386
2,19221,0.003097,0.047720,0.037898,0.000347,0.002728,0.309697,0.476866
3,19222,0.159511,0.002122,0.000629,0.000838,0.012501,0.370600,0.433185
4,19223,0.003568,0.002735,0.000491,0.002106,0.006211,0.667946,0.423215
...,...,...,...,...,...,...,...,...
12809,32028,0.063707,0.117779,0.001965,0.000216,0.023094,0.189167,0.338225
12810,32029,0.159389,0.003001,0.028947,0.007801,0.147706,0.175749,0.413261
12811,32030,0.001394,0.000836,0.948871,0.000139,0.000546,0.001671,0.064532
12812,32031,0.325968,0.006518,0.015308,0.000098,0.040120,0.194836,0.348595


In [55]:
submission.to_csv('lgb_with_train_data.csv', index=False)